### Prompt Chaining and Sequencing Tutorial
#### Overview
This tutorial explores the concepts of prompt chaining and sequencing in the context of working with large language models. We'll use OpenAI's GPT models and the LangChain library to demonstrate how to connect multiple prompts and build logical flows for more complex AI-driven tasks.

#### Motivation
As AI applications become more sophisticated, there's often a need to break down complex tasks into smaller, manageable steps. Prompt chaining and sequencing allow us to guide language models through a series of interrelated prompts, enabling more structured and controlled outputs. This approach is particularly useful for tasks that require multiple stages of processing or decision-making.

In [1]:
import os
import re

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

#### Basic Prompt Chaining

In [2]:
# Define prompt templates
story_prompt = PromptTemplate(
    input_variables=["genre"],
    template="Write a short {genre} story in 3-4 sentences."
)

summary_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize the following story in one sentence:\n{story}"
)

# Chain the prompts
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    story = (story_prompt | llm).invoke({"genre": genre}).content
    summary = (summary_prompt | llm).invoke({"story": story}).content
    return story, summary

# Test the chain
genre = "science fiction"
story, summary = story_chain(genre)
print(f"Story: {story}\n\nSummary: {summary}")

Story: In the year 2147, humanity discovered a way to communicate with alternate realities through a device called the Convergence Sphere. When a young scientist named Lira activated the Sphere, she received a message from a version of herself living in a world ravaged by perpetual war. Driven by a deep sense of empathy, she shared insights on peace and unity, unknowingly setting off a chain reaction that began to heal the fractures in both realities. As the timeline converged, Lira found herself standing at the crossroads of existence, where every choice could save or shatter worlds.

Summary: In 2147, young scientist Lira activates the Convergence Sphere, leading her to communicate with an alternate version of herself in a war-torn reality, and her insights on peace initiate a healing process for both worlds as she faces the weight of choices that could either save or destroy them.


#### Sequential Prompting

In [3]:
# Define prompt templates for each step
theme_prompt = PromptTemplate(
    input_variables=["text"],
    template="Identify the main theme of the following text:\n{text}"
)

tone_prompt = PromptTemplate(
    input_variables=["text"],
    template="Describe the overall tone of the following text:\n{text}"
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"
)

def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    theme = (theme_prompt | llm).invoke({"text": text}).content
    tone = (tone_prompt | llm).invoke({"text": text}).content
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme of the text is the duality of artificial intelligence advancements, highlighting both the transformative potential of AI and the ethical concerns it raises, emphasizing the need for cautious and responsible development.

Tone: The overall tone of the text is cautious yet hopeful. It acknowledges the excitement surrounding the advancements in artificial intelligence while simultaneously highlighting the concerns and ethical implications that arise. The emphasis on the need for careful consideration and foresight suggests a balanced approach to AI development, aiming to maximize benefits while addressing potential risks.

Takeaways: Here are the key takeaways from the text:

1. **Transformative Potential**: AI has the capacity to revolutionize various industries and enhance everyday life, suggesting significant benefits that can arise from its advancement.

2. **Ethical Concerns**: The development of AI brings forth important ethical issues, including concerns about

#### Dynamic Prompt Generation

In [4]:
# Define prompt templates
answer_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely:\n{question}"
)

follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []
    current_question = initial_question

    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        qa_chain.append({"question": current_question, "answer": answer})
        
        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain

# Test the dynamic Q&A system
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)

for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: Potential applications of quantum computing include:

1. **Cryptography**: Breaking traditional encryption methods and enhancing secure communications through quantum key distribution.
2. **Drug Discovery**: Simulating molecular interactions and chemical reactions for faster drug development.
3. **Optimization Problems**: Solving complex optimization challenges in logistics, finance, and operations research more efficiently.
4. **Machine Learning**: Improving algorithms for data analysis and pattern recognition.
5. **Materials Science**: Designing new materials with tailored properties at the atomic level.
6. **Financial Modeling**: Enhancing risk analysis and portfolio optimization.
7. **Climate Modeling**: Improving simulations for climate change and environmental modeling.
8. **Artificial Intelligence**: Accelerating training and inference for AI models.

These applications leverage the unique properties of quantum me

#### Error Handling and Validation

In [6]:
# Define prompt templates
generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)
    return match.group() if match else None

def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):
        try:
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            number = extract_number(response)
            
            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")
            
            # Validate the relevance
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
    
    return "Failed to generate a valid number after multiple attempts."

# Test the robust number generation
topic = "World War II"
result = robust_number_generation(topic)
print(f"Final result for topic '{topic}': {result}")


Final result for topic 'World War II': 1945
